In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from importlib import reload
import curses  ####  pip install windows-curses

import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

import specsanalyzer
from specsanalyzer import SpecsAnalyzer

In [ ]:
# reload(specsanalyzer)
spa = SpecsAnalyzer()

In [ ]:
with open("../tests/data/dataEPFL/R9132/Data9132_RAWDATA.tsv") as file:
    # with open('../tests/data/dataFHI/Scan1232.tsv') as file:
    tsv_data = np.loadtxt(file, delimiter="\t")
plt.figure()
plt.imshow(tsv_data)

In [ ]:
# Testing convert and crop on the imported data
spa = SpecsAnalyzer(config="../tests/data/dataEPFL/config/config.yaml")

lens_mode = "WideAngleMode"
kinetic_energy = 35.0
pass_energy = 35.0

res_xarray = spa.convert_image(
    tsv_data, pass_energy, kinetic_energy, lens_mode
)
plt.figure()
res_xarray.plot(cmap="viridis", vmin=0)

In [ ]:
with open("../tests/data/dataEPFL/R9132/Data9132_IGOR_corrected.tsv") as file:
    # with open('../tests/data/dataFHI/Scan1232_converted.tsv') as file:
    ref_data = np.loadtxt(file, delimiter="\t")
    ref_data = ref_data
    # ref_data = ref_data.T
# angle_axis = np.linspace(-18, 17.3077, ref_data.shape[0])
# angle_axis = np.linspace(-18, 17.3077, ref_data.shape[0])

# ek_axis = np.linspace(15.86, 21.1247, ref_data.shape[1])
# ek_axis = np.linspace(33.85, 37.1404, ref_data.shape[1])


import specsanalyzer.convert

# reload(specsanalyzer.convert)
# let's get all the functions to be tested
from specsanalyzer.convert import get_damatrix_fromcalib2d
from specsanalyzer.convert import get_rr_da
from specsanalyzer.convert import calculate_polynomial_coef_da
from specsanalyzer.convert import mcp_position_mm
from specsanalyzer.convert import calculate_matrix_correction

spa = SpecsAnalyzer(config="../tests/data/dataEPFL/config/config.yaml")
config_dict = spa.return_config()
lens_mode = "WideAngleMode"
kinetic_energy = 35
pass_energy = 35
binning = 4

(
    ek_axis,
    angle_axis,
    angular_correction_matrix,
    e_correction,
    jacobian_determinant,
) = calculate_matrix_correction(
    lens_mode,
    pass_energy,
    kinetic_energy,
    binning,
    config_dict,
)


ref_xarray = xr.DataArray(
    data=ref_data,
    coords={"Angle": angle_axis, "Ekin": ek_axis},
    dims=["Angle", "Ekin"],
)
plt.figure()
ref_xarray.plot()

In [ ]:
# ref_xarray_reint=ref_xarray.interp_like(res_xarray)
# ff_xarray = (res_xarray - ref_xarray_reint)/

normalization = np.abs((ref_xarray.to_numpy())).mean()

# ff_xarray = (res_xarray - ref_xarray)/normalization
ff_xarray = (res_xarray - ref_xarray) / ref_xarray

np.abs((ff_xarray.to_numpy())).sum()

In [ ]:
plt.figure()
ff_xarray.plot(cmap="RdBu_r", vmax=1e-5)
plt.title("Difference/Mean pixel intensity")

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=(4, 8))
subplot1 = ref_xarray.plot(ax=ax1, cmap="viridis", vmin=0)
subplot2 = res_xarray.plot(ax=ax2, cmap="viridis", vmin=0)
subplot3 = ff_xarray.plot(cmap="RdBu_r", ax=ax3, vmax=100)

plt.tight_layout()

In [ ]:

lens_mode="WideAngleMode"
kinetic_energy=35.0
pass_energy=35.0

%timeit res_xarray = spa.convert_image(tsv_data, pass_energy, kinetic_energy, lens_mode)

In [ ]:
ref_xarray

In [ ]:
print(angle_axis[0], angle_axis[angle_axis.size - 1])

In [ ]:
from scipy import interpolate

x = np.arange(-5.01, 5.01, 0.25)

y = np.arange(-5.01, 5.01, 0.3)

xx, yy = np.meshgrid(x, y)

z = np.sin(xx**2 + yy**2)

f = interpolate.interp2d(x, y, z, kind="cubic")

In [ ]:
import matplotlib.pyplot as plt

xnew = np.arange(-5.01, 5.01, 1e-2)

ynew = np.arange(-5.01, 5.01, 2e-2)

znew = f(xnew, ynew)

plt.contourf(xnew, ynew, znew)

plt.show()

In [ ]:
import numpy as np
from scipy.interpolate import RectBivariateSpline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Regularly-spaced, coarse grid
dx, dy = 0.4, 0.4
xmax, ymax = 2, 4
x = np.arange(-xmax, xmax, dx)
y = np.arange(-ymax, ymax, dy)
X, Y = np.meshgrid(x, y)
Z = np.exp(-((2 * X) ** 2) - (Y / 2) ** 2)

interp_spline = RectBivariateSpline(y, x, Z)

# Regularly-spaced, fine grid
dx2, dy2 = 0.16, 0.16
x2 = np.arange(-xmax, xmax, dx2)
y2 = np.arange(-ymax, ymax, dy2)
X2, Y2 = np.meshgrid(x2, y2)
Z2 = interp_spline(y2, x2)

fig, ax = plt.subplots(nrows=1, ncols=2, subplot_kw={"projection": "3d"})
ax[0].plot_wireframe(X, Y, Z, color="k")

ax[1].plot_wireframe(X2, Y2, Z2, color="k")
for axes in ax:
    axes.set_zlim(-0.2, 1)
    axes.set_axis_off()

fig.tight_layout()
plt.show()

In [ ]:
from audioop import minmax
from interpolation.splines import eval_linear
from interpolation.splines import UCGrid, CGrid, nodes

minx = 0.0
maxx = 344.0
steps = 345
grid = UCGrid((minx, maxx, steps), (-1.0, 1.0, 10))
grid